# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [3]:
import pandas as pd
recipes = pd.read_csv('/content/recipes_sample.csv')

In [50]:
titles = ['id   ', ' minutes ']
print('|{:>7}|{:>8}|'.format(*titles))
print(f'-'*19)
for j in range(5):
    print(f"|{recipes['id'][j]: ^ 7}|{recipes['minutes'][j]: ^ 9}|")

|  id   | minutes |
-------------------
| 44123 |    90   |
| 67664 |    10   |
| 38798 |    30   |
| 35173 |    45   |
| 84797 |    25   |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [64]:
from xml.etree import ElementTree as ET

with open('steps_sample.xml', 'r') as f:
    tree = ET.parse(f)

steps_sample = {}
for recipe in tree.findall('recipe'):
    recipe_id = recipe.find('id').text
    steps = [step.text for step in recipe.findall('step')]
    steps_sample[recipe_id] = steps

def show_info(recipe_id):
    a = recipes[recipes.id == recipe_id]
    inf = f'"{a.name.values[0]}"'.title()
    inf += '\n\n'
    steps = steps_sample.get(str(recipe_id), [])
    for i, step in enumerate(steps, start=1):
        inf += f'{i}. {step.capitalize()}\n'
    inf += '-'*10 + '\n'
    inf += f'Автор: {a.contributor_id.values[0]}\n'
    inf += f'Среднее время приготовления: {a.minutes.values[0]} минут'
    
    return inf

print(show_info(170895))

"Leeks And Parsnips  Sauteed Or Creamed"

----------
Автор: 8377
Среднее время приготовления: 27 минут


## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [73]:
from bs4 import BeautifulSoup as bs

with open('steps_sample.xml', 'r') as f:
    xml_content = f.read()
    bs_content = bs(xml_content, 'xml')
    
for recipe in bs_content.find_all('recipe'):
    if recipe.id.text == '25082':
        steps_list = list(recipe.find_all('step'))
        
for step in steps_list:
    if re.search(r'\d+ hour|hours|\d+ minute|minutes', str(step)):
        print(step.get_text())

turn out onto a lightly floured board and knead for about 20 minutes , adding flour as nescessary to keep the dough from sticking to the board
when it has been sufficiently kneaded , cover it with a damp cloth for about 10 minutes and wash and grease the bowl lightly
let the dough rise until it springs back when you stick your finger in it , and it is about twice the size as it was before (this takes about 2 hours
when the dough has risen twice , deflate it again and cover it with the damp cloth again for about 10 minutes , then divide it into and shape it into loaves , buns , etc
let the shaped dough rise on a baking sheet or in a loaf-pan for about an hour maybe- it will rise a bit more in the oven
bake at 400 for 20 minutes , and then turn the oven down to 350 and bake for 20-30 minutes longer , until the loaf is a lovely brown and sounds hollow when you thump it on the bottom


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [88]:
pattern = re.compile(r'^this(?:[\w]|[ ])*,[ ]?but')
x = []
o = 0
for i in recipes.description.values:
    result = pattern.match(str(i))
    if result:
        k+=1
        x.append(i)

for j in rn.sample(x, 3):
    print(j, end=f'\n{"-"*127}\n')

print(o)

this is pretty easy, but oh-so-fancy. great for a girl party.
-------------------------------------------------------------------------------------------------------------------------------
this is a great meal eaten the same day ,but even better the next day , if you can wait! add your favourite spices, but try it first as it is and i think that you will enjoy the 'vegetable' taste. good for freezing.
-------------------------------------------------------------------------------------------------------------------------------
this has a lot of ingredients, but is totally easy to make.  an excellent home cooked meal that tastes like chinese take-out!  you can also make these as appetizers (as they are served at the restaurants such as pf changs).
-------------------------------------------------------------------------------------------------------------------------------
0


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [104]:
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.tokenize import sent_tokenize
import sys
from nltk import pos_tag
from nltk import word_tokenize
import nltk.data

In [112]:
nltk.download('unit')

tt = ToktokTokenizer()
unique = set()

for key, value in steps_sample.items():
    unique.update(tt.tokenize(i.lower()) for i in value if i.isalpha())

alst = list(unique)

[nltk_data] Error loading unit: Package 'unit' not found in index


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [115]:
from nltk.tokenize import sent_tokenize

df = pd.read_csv("recipes_sample.csv").astype(str)
id_length = {}

for index, row in df.iterrows():
    id = str(row['id'])
    description = row['description'].lower()
    sentences = sent_tokenize(description)
    length = len(sentences)
    id_length[id] = length

sorted_tuples = sorted(id_length.items(), key=lambda x: x[1], reverse=True)
sorted_tuples[:5]

[('334113', 76),
 ('287008', 27),
 ('328708', 24),
 ('205348', 23),
 ('316000', 23)]

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [116]:
new_recipes = recipes.loc[recipes['id'] == 241106]
name = new_recipes['name'].values[0]
print(stag(name))

    JJ     NNS    IN     NNS    VBP    JJ   CC   JJ    NNS  
eggplant steaks with chickpeas feta cheese and black olives 
